In [1]:

import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import io

In [2]:
filepath_train_en = '../data/raw/train.en' 
filepath_train_fr = '../data/raw/train.fr'

In [3]:
try:
    en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
    fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')
except OSError:
    print("chưa tải được gói ngôn ngữ!")

In [4]:
def yield_tokens(file_path, tokenizer):
    with io.open(file_path, encoding='utf-8') as f:
        for line in f:
            yield tokenizer(line.strip())

In [5]:

special_tokens = ['<unk>', '<pad>', '<sos>', '<eos>']


vocab_en = build_vocab_from_iterator(
    yield_tokens(filepath_train_en, en_tokenizer),
    min_freq=2,
    specials=special_tokens,
    max_tokens=10000     
)

In [6]:
vocab_en.set_default_index(vocab_en['<unk>'])

vocab_fr = build_vocab_from_iterator(
    yield_tokens(filepath_train_fr, fr_tokenizer),
    min_freq=2,
    specials=special_tokens,
    max_tokens=10000
)
vocab_fr.set_default_index(vocab_fr['<unk>'])

In [7]:
PAD_IDX = vocab_en['<pad>']
SOS_IDX = vocab_en['<sos>']
EOS_IDX = vocab_en['<eos>']

In [8]:
def text_transform(tokenizer, vocab, text):
    token_list = tokenizer(text.strip())
    index_list = [vocab[token] for token in token_list]
    return torch.tensor([SOS_IDX] + index_list + [EOS_IDX])

In [9]:
def collate_batch(batch):
    src_batch, trg_batch = [], []
    src_lens = []

    for src_sample, trg_sample in batch:
        # Biến đổi văn bản thô thành tensor số
        src_item = text_transform(en_tokenizer, vocab_en, src_sample)
        trg_item = text_transform(fr_tokenizer, vocab_fr, trg_sample)
        
        src_batch.append(src_item)
        trg_batch.append(trg_item)
        # Lưu lại độ dài thật của câu tiếng Anh (để dùng cho pack_padded_sequence)
        src_lens.append(len(src_item))

    # --- BẮT BUỘC: Sắp xếp batch theo độ dài giảm dần ---
    #
    # Lý do: PyTorch yêu cầu input của packing phải được sort trước
    zipped = list(zip(src_batch, trg_batch, src_lens))
    # Sắp xếp dựa trên src_lens (phần tử thứ 2 trong tuple) từ cao xuống thấp
    zipped.sort(key=lambda x: x[2], reverse=True)
    
    # Tách ngược trở lại thành các list riêng lẻ
    src_batch, trg_batch, src_lens = zip(*zipped)
    
    # Chuyển src_lens sang tensor
    src_lens = torch.tensor(src_lens)

    # --- PADDING: Điền thêm <pad> vào câu ngắn ---
    # padding_value=PAD_IDX: Điền số 1 vào chỗ trống
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX)

    return src_batch, trg_batch, src_lens

In [10]:
BATCH_SIZE = 64  #

# Đọc dữ liệu thô từ file vào list (để đưa vào DataLoader)
def read_raw_data(path_en, path_fr):
    with open(path_en, encoding='utf-8') as f_en, open(path_fr, encoding='utf-8') as f_fr:
        return list(zip(f_en, f_fr))


train_data = read_raw_data(filepath_train_en, filepath_train_fr)

train_loader = DataLoader(
    train_data, 
    batch_size=BATCH_SIZE, 
    collate_fn=collate_batch,
    shuffle=True # Nên xáo trộn dữ liệu khi train
)

In [11]:
print("\n=== KIỂM TRA DATALOADER (PADDING & PACKING) ===")
src, trg, src_len = next(iter(train_loader))

print(f"✅ Kích thước Source Batch: {src.shape}")
print(f"   (Dài nhất trong batch x Batch Size)")
print(f"✅ Kích thước Target Batch: {trg.shape}")
print(f"✅ Danh sách độ dài (đã sắp xếp giảm dần chưa?):")
print(src_len) # In ra xem có phải là 1 dãy số giảm dần không (VD: 20, 19, 15, 10...)


=== KIỂM TRA DATALOADER (PADDING & PACKING) ===
✅ Kích thước Source Batch: torch.Size([24, 64])
   (Dài nhất trong batch x Batch Size)
✅ Kích thước Target Batch: torch.Size([26, 64])
✅ Danh sách độ dài (đã sắp xếp giảm dần chưa?):
tensor([24, 24, 23, 23, 23, 20, 20, 20, 20, 19, 19, 19, 18, 18, 18, 18, 17, 17,
        17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 12,
        12, 12, 12, 12, 11, 11, 11, 10,  9,  9])
